### Required
- Create conda environment
- pip install -r requirements.txt

In [72]:
import pandas as pd
import os
os.environ["HF_HOME"] = "/data2/model_weights/huggingface"
from transformers import pipeline
import torch
import re
import json

In [ ]:
# placeholder
df = pd.read_csv(YOUR_DATA_HERE)

In [2]:
model = "Qwen/Qwen2.5-7B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda:1",
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.06s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [64]:
# Plenty of directions to take this.

prompt = """
You are a helpful pharmaceutical assistant. Extract the total daily dosage from the following dosage instructions. Format the dosage as a json with strength and frequency fields. Explain how you calculated the daily_sumamry.

Hints:
- When the number of tablets is given rather than the mass, convert the strength to mass.

Example: 
1. 'Metformin 20mg tablets. take 0.25 tablet by mouth 2 (two) times daily' = {"events": [{"frequency": "twice daily","amount": {"value": 0.25,"unit": "tablet"},"strength": {"value": 5, "unit": "mg"}}],"daily_summary": {"total_amount": {"value": 0.5,"unit": "tablet"},"total_strength": {"value": 10, "unit": "mg"}}}
2. 'Tylenol 50mg tablets. take 1 tablet daily and 2 tablets nightly by mouth.' {"events": [{"frequency": "daily","amount": {"value": 1,"unit": "tablet"},"strength": {"value": 50,"unit": "mg"}},{"frequency": "nightly","amount": {"value": 2,"unit": "tablet"},"strength": {"value": 50,"unit": "mg"}}],"daily_summary": {"total_amount": {"value": 3,"unit": "tablets"},"total_strength": {"value": 150,"unit": "mg"}}}

Dosage Instruction:
"""

# messages = [
#     {"role": "system", "content": prompt},
#     {
#         "role": "user",
#         "content": "Albuterol 50mg. take 0.5 tablet by mouth 2 (two) times daily",
#     },
# ]

In [90]:
# for dosage_instruction in df.dosage_text:
responses = []
for dosage_instruction in ["Albuterol 50mg. take 0.5 tablet by mouth 2 (two) times daily"]:
    prompted = pipe(
        prompt + dosage_instruction,
        do_sample=False,
        temperature=1.0,
        top_p=1,
        max_new_tokens=300
    )
    print(prompted[0]["generated_text"])
    str_resp = re.search("```json(.*)(?=```)", prompted[0]["generated_text"], re.DOTALL).group(1)
    responses.append(json.loads(str_resp))

/home/nea914/miniconda3/envs/unsloth/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



You are a helpful pharmaceutical assistant. Extract the total daily dosage from the following dosage instructions. Format the dosage as a json with strength and frequency fields. Explain how you calculated the daily_sumamry.

Hints:
- When the number of tablets is given rather than the mass, convert the strength to mass.

Example: 
1. 'Metformin 20mg tablets. take 0.25 tablet by mouth 2 (two) times daily' = {"events": [{"frequency": "twice daily","amount": {"value": 0.25,"unit": "tablet"},"strength": {"value": 5, "unit": "mg"}}],"daily_summary": {"total_amount": {"value": 0.5,"unit": "tablet"},"total_strength": {"value": 10, "unit": "mg"}}}
2. 'Tylenol 50mg tablets. take 1 tablet daily and 2 tablets nightly by mouth.' {"events": [{"frequency": "daily","amount": {"value": 1,"unit": "tablet"},"strength": {"value": 50,"unit": "mg"}},{"frequency": "nightly","amount": {"value": 2,"unit": "tablet"},"strength": {"value": 50,"unit": "mg"}}],"daily_summary": {"total_amount": {"value": 3,"unit"

In [91]:
responses

[{'events': [{'frequency': 'twice daily',
    'amount': {'value': 0.5, 'unit': 'tablet'},
    'strength': {'value': 50, 'unit': 'mg'}}],
  'daily_summary': {'total_amount': {'value': 1, 'unit': 'tablet'},
   'total_strength': {'value': 50, 'unit': 'mg'}}}]